In [4]:
import altair as alt
#from altair import pipe, limit_rows, to_values
import altair_viewer
'''t = lambda data: pipe(data, limit_rows(max_rows=10000), to_values)
alt.data_transformers.register('custom', t)
alt.data_transformers.enable('custom')'''
alt.data_transformers.disable_max_rows()
altair_viewer._global_viewer._use_bundled_js = False
alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

In [21]:
df = pd.read_csv('Superstore.csv', encoding='windows-1252')
c = alt.Chart(df).mark_bar().encode(
    x=str('State:N'),
    y=('median(Quantity):Q'),
    color=str('Category:N')
).facet(column=str('Region:N')
).resolve_scale(x = 'independent')
c

alt.FacetChart(...)

In [75]:
import altair as alt
import pandas as pd
import altair_viewer
alt.data_transformers.disable_max_rows()
altair_viewer._global_viewer._use_bundled_js = False
alt.data_transformers.enable('data_server')

Measure = ['Quantity','Profit','Discount','Sales']



def plotBar(row,column,mes,dimen):         
    print(row,column)
    if dimen == 1:     #column 1 , row 1
        print('1 Dimension')
        if mes == 'row':               #Measure in Row
            sy = str(row[1]+'('+row[0]+')')
        else:
            if type(row[0]) == type(['datetime']):
            #if df[row[0]].dtypes == 'datetime64[ns]':   #Dimension(Datetime) in Row
                sy = str(row[0][1]+'('+row[0][0]+')')
            else:
                sy = str(row[0])                        #normal Dimension in Row

        if mes == 'column':           #Measure in Column
            sx = str(column[1]+'('+column[0]+')')
        else:
            if type(column[0]) == type(['datetime']):
            #if df[column[0]].dtypes == 'datetime64[ns]':    #Dimension(Datetime) in column
                sx = str(column[0][1]+'('+column[0][0]+')')
            else:
                sx = str(column[0])                     #normal Dimension in Column
        
        c = alt.Chart(df).mark_bar().encode(
            x=sx,
            y=sy,
            tooltip = [sy,sx]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c
    elif dimen == 2:
        print('2 Dimension')                                  #2 Dimension
        if mes == 'row' :               #2 Column (Dimension)
            Di = column
            Me = row[0]
            fil = row[1]

            if (type(Di[-1]) == type(['datetime'])) and (type(Di[-2]) == type(['datetime'])):
                scol = str(Di[-2][1]+'('+Di[-2][0]+')')           
                sx = str(Di[-1][1]+'('+Di[-1][0]+')')
            elif type(Di[-2]) == type(['datetime']):
                scol = str(Di[-2][1]+'('+Di[-2][0]+')')           
                sx = str(Di[-1])
            elif type(Di[-1]) == type(['datetime']):
                scol = str(Di[-2])
                sx = str(Di[-1][1]+'('+Di[-1][0]+')')
            else:
                scol = str(Di[-2])                      #column > x
                sx = str(Di[-1])

            c = alt.Chart(df).mark_bar().encode(
                x=sx,
                y=str(fil+'('+Me+')'),
                #color=scol,
                tooltip = [sx,str(fil+'('+Me+')')]
            ).facet(column=scol
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        elif mes == 'column':                 #2 Row (Dimension)
            Di = row    
            Me = column[0]
            fil = column[1]

            if (type(Di[-1]) == type(['datetime'])) and (type(Di[-2]) == type(['datetime'])):
                srow = str(Di[-2][1]+'('+Di[-2][0]+')')           
                sy = str(Di[-1][1]+'('+Di[-1][0]+')')
            elif type(Di[-2]) == type(['datetime']):            #year,date error (large data)
                srow = str(Di[-2][1]+'('+Di[-2][0]+')')
                sy = str(Di[-1])
            elif type(Di[-1]) == type(['datetime']):
                srow = str(Di[-2])
                sy = str(Di[-1][1]+'('+Di[-1][0]+')')
            else:
                srow = str(Di[-2])
                sy = str(Di[-1])
            c = alt.Chart(df).mark_bar().encode(
                x=str(fil+'('+Me+')'),
                y=sy,
                #color=srow,
                tooltip = [sy,str(fil+'('+Me+')')]
            ).facet(row=srow
            ).resolve_scale(y = 'independent',x = 'independent')
            return c
    elif dimen == 3:   #3 Dimension
        print('3 Dimension')
        if mes == 'row' :               #2 Column (Dimension)
            Di = column
            Me = row[0][0]
            fil = row[0][1]

            if (type(Di[-1]) == type(['datetime'])) and (type(Di[-2]) == type(['datetime'])) and (type(Di[-3]) == type(['datetime'])):
                scol = str(Di[-3][1]+'('+Di[-3][0]+')')
                sx = str(Di[-2][1]+'('+Di[-2][0]+')')
                scolor = str(Di[-1][1]+'('+Di[-1][0]+')')
            elif (type(Di[-1]) == type(['datetime'])) and (type(Di[-2]) == type(['datetime'])):
                scol = str(Di[-3])
                sx = str(Di[-2][1]+'('+Di[-2][0]+')')
                scolor = str(Di[-1][1]+'('+Di[-1][0]+')')
            elif (type(Di[-1]) == type(['datetime'])) and (type(Di[-3]) == type(['datetime'])):
                scol = str(Di[-3][1]+'('+Di[-3][0]+')')
                sx = str(Di[-2])
                scolor = str(Di[-1][1]+'('+Di[-1][0]+')')
            elif (type(Di[-2]) == type(['datetime'])) and (type(Di[-3]) == type(['datetime'])):
                scol = str(Di[-3][1]+'('+Di[-3][0]+')')
                sx = str(Di[-2][1]+'('+Di[-2][0]+')')
                scolor = str(Di[-1])
            elif type(Di[-2]) == type(['datetime']):          
                scol = str(Di[-3])
                sx = str(Di[-2][1]+'('+Di[-2][0]+')')
                scolor = str(Di[-1])
            elif type(Di[-1]) == type(['datetime']):
                scol = str(Di[-3])
                sx = str(Di[-2])
                scolor = str(Di[-1][1]+'('+Di[-1][0]+')')
            elif type(Di[-3]) == type(['datetime']):
                scol = str(Di[-3][1]+'('+Di[-3][0]+')')
                sx = str(Di[-2])
                scolor = str(Di[-1])
            else:
                scol = str(Di[-3])                      #column > x > color
                sx = str(Di[-2])
                scolor = str(Di[-1])
            c = alt.Chart(df).mark_bar().encode(
                x=sx,
                y=str(fil+'('+Me+')'),
                color=scolor,
                tooltip = [scolor,sx,str(fil+'('+Me+')')]
            ).facet(column=scol
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        elif mes == 'column':                 #2 Row (Dimension)
            Di = row    
            Me = column[0][0]
            fil = column[0][1]

            if (type(Di[-1]) == type(['datetime'])) and (type(Di[-2]) == type(['datetime'])) and (type(Di[-3]) == type(['datetime'])):
                srow = str(Di[-3][1]+'('+Di[-3][0]+')')
                sy = str(Di[-2][1]+'('+Di[-2][0]+')')
                scolor = str(Di[-1][1]+'('+Di[-1][0]+')')
            elif (type(Di[-1]) == type(['datetime'])) and (type(Di[-2]) == type(['datetime'])):
                srow = str(Di[-3])
                sy = str(Di[-2][1]+'('+Di[-2][0]+')')
                scolor = str(Di[-1][1]+'('+Di[-1][0]+')')
            elif (type(Di[-1]) == type(['datetime'])) and (type(Di[-3]) == type(['datetime'])):
                srow = str(Di[-3][1]+'('+Di[-3][0]+')')
                sy = str(Di[-2])
                scolor = str(Di[-1][1]+'('+Di[-1][0]+')')
            elif (type(Di[-2]) == type(['datetime'])) and (type(Di[-3]) == type(['datetime'])):
                srow = str(Di[-3][1]+'('+Di[-3][0]+')')
                sy = str(Di[-2][1]+'('+Di[-2][0]+')')
                scolor = str(Di[-1])
            elif type(Di[-2]) == type(['datetime']):        
                srow = str(Di[-3])
                sy = str(Di[-2][1]+'('+Di[-2][0]+')')
                scolor = str(Di[-1])
            elif type(Di[-1]) == type(['datetime']):
                srow = str(Di[-3])
                sy = str(Di[-2])
                scolor = str(Di[-1][1]+'('+Di[-1][0]+')')
            elif type(Di[-3]) == type(['datetime']):
                srow = str(Di[-3][1]+'('+Di[-3][0]+')')
                sy = str(Di[-2])
                scolor = str(Di[-1])
            else:
                srow = str(Di[-3])                      #column > x > color
                sy = str(Di[-2])
                scolor = str(Di[-1])
            c = alt.Chart(df).mark_bar().encode(
                x=str(fil+'('+Me+')'),
                y=sy,
                color=scolor,
                tooltip = [scolor,sy,str(fil+'('+Me+')')]
            ).facet(row=srow
            ).resolve_scale(y = 'independent',x = 'independent')
            return c
        
def plotLine(row,column,mes,Dimen):

    print(row,column)
    if mes == 'column':                 #column is Measure
        fil = column[1]
        Me = column[0]
        if Dimen == 1:
            Di = row[0][0]
            fd = row[0][1]
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.X(str(fil+'('+Me+'):Q')),
                alt.Y(str(fd+'('+Di+'):T')),
                tooltip = [str(fd+'('+Di+'):T'),str(fil+'('+Me+'):Q')]
            )
            return ch
        elif Dimen == 2:
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.X(str(fil+'('+Me+'):Q')),
                alt.Y(str(row[-1][1]+'('+row[-1][0]+'):T')),
                row = str(row[-2][1]+'('+row[-2][0]+'):T'),
                tooltip = [str(row[-1][1]+'('+row[-1][0]+'):T'),str(fil+'('+Me+')')]
            ).resolve_legend(
                size='independent'
            ).resolve_scale(
                y = 'independent',x = 'independent'
            )
            return ch

    elif mes == 'row':   #row is Measure
        Me = row[0]
        fil = row[1]
        #elif df[c].dtypes == 'datetime64[ns]':
        if Dimen == 1:
            Di = column[0][0]
            fd = column[0][1]
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.X(str(fd+'('+Di+'):T')),
                alt.Y(str(fil+'('+Me+'):Q')),
                tooltip = [str(fd+'('+Di+'):T'),str(fil+'('+Me+'):Q')]
            )
            return ch
        elif Dimen == 2:
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.Y(str(fil+'('+Me+'):Q')),
                alt.X(str(column[-1][1]+'('+column[-1][0]+'):T')),
                column = str(column[-2][1]+'('+column[-2][0]+'):T'),
                tooltip = [str(column[-1][1]+'('+column[-1][0]+'):T'),str(fil+'('+Me+')')]
            ).resolve_legend(
                size='independent'
            ).resolve_scale(
                y = 'independent',x = 'independent'
            )
            return ch

def plotPie(row,column,mes):

    fd = 'year'                                     ####year
    if mes == 'row':
        Mes = row[0]
        fil = row[1]
        Di = column[0]
    elif mes == 'column':
        Mes = column[0]
        fil = column[1]
        Di = row[0]

    if type(Di) == type([]):
        s = str(Di[1]+'('+Di[0]+'):T')
    else:
        s = str(Di+':N')

    base = alt.Chart(df).mark_arc().encode(
        theta=alt.Theta(str(fil+'('+Mes+'):Q')), 
        color=alt.Color(s, type="nominal"), 
        tooltip = [s,str(fil+'('+Mes+'):Q')]
    )

    return base

def plotGraph(row,column,tp):

    def checkMeasure(R,C):      #True when row is measure
        for r in R:
            if type(r) == type([]):
                if r[0] in Measure:
                    return True
                else:
                    return False
            else:
                return False

    if tp == 'line':
        if checkMeasure(row,column):    #row is measure
            print('row is measurement')
            chart = []
            for r in row:
                chart.append(plotLine(r,column,'row',len(column)))
            return alt.vconcat(*chart)
        else:                       #column is Measurement
            print('column is measurement')
            chart = []
            for c in column:
                chart.append(plotLine(row,c,'column',len(row)))
            return alt.hconcat(*chart)

    elif tp == 'bar':
        if checkMeasure(row,column):    #row is measure
            print('row is measurement')
            if len(column) > 2:
                print('column 3 Dimen')
                return plotBar(row,column,'row',len(column))
            chart = []
            for r in row:
                chart.append(plotBar(r,column,'row',len(column)))
            return alt.vconcat(*chart)
        else:
            print('column is measurement')
            if len(row) > 2:
                print('row is 3 Dimen')
                return plotBar(row,column,'column',len(row))
            chart = []
            for c in column:
                chart.append(plotBar(row,c,'column',len(row)))
            return alt.hconcat(*chart)

    elif tp == 'pie':
        if checkMeasure(row,column):    #row is measure
            print('row is measurement')
            chart = []
            for r in row:
                chart.append(plotPie(r,column,'row'))
            return alt.vconcat(*chart)
        else:
            print('column is measurement')
            chart = []
            for c in column:
                chart.append(plotPie(row,c,'column'))
            return alt.hconcat(*chart)

df = pd.read_csv('Superstore.csv', encoding='windows-1252')
df['Order Date'] = pd.to_datetime(df['Order Date'],format='%d/%m/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'],format='%d/%m/%Y')

fd1 = 'date'
fd2 = 'month'
fd3 = 'year'

D1 = ['Order Date','month']
D2 = ['Ship Date','year']

D3 = 'Category'
D4 = 'Sub-Category'

D5 = 'Country/Region'
D6 = 'Region'
D7 = 'State'
D8 = 'City'

D9 = 'Ship Mode'

M1 = ['Quantity','mean']
M2 = ['Discount','sum']
M3 = ['Profit','max']
M4 = ['Sales','count']

row = [D2]
column = [M4,M3,M2,M1]

column , row = row ,column

#plotBar(row,column,f1)
#plotLine(row,column,f1)
#plotPie(row,column,f1)
plotGraph(row,column,'line')
    

row is measurement
['Sales', 'count'] [['Ship Date', 'year']]
['Profit', 'max'] [['Ship Date', 'year']]
['Discount', 'sum'] [['Ship Date', 'year']]
['Quantity', 'mean'] [['Ship Date', 'year']]


alt.VConcatChart(...)

In [ ]:
import pandas as pd
import json

df = pd.read_csv('SS_20lines.csv', encoding='windows-1252')
data = {'dataframe':df,'row':[],'column':[],'typegraph':''}
def createjson() :
    data_json = json.dumps(data)
    jsonFile = open("data.json","w")
    jsonFile.write(data_json)
    jsonFile.close()
    with open ('data.json', encoding = 'utf8' ) as data_json :
        data_read = data_json.read()
    data_read 

js = df.to_json(orient = 'columns')
df2 = pd.read_json(js)
#createjson()

In [16]:
import altair as alt
from vega_datasets import data
import pandas as pd


df = pd.read_csv('Superstore.csv', encoding='windows-1252')

df['Order Date'] = pd.to_datetime(df['Order Date'],format='%d/%m/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'],format='%d/%m/%Y')

itemtest = ['sum(Quantity):Q','sum(Profit):Q']
charts = []
theta_charts = []
for i in itemtest:
    subChart = alt.Chart(df).mark_arc().encode(
    alt.Theta(i),
    alt.Tooltip([i]),
)
    charts.append(subChart)
alt.vconcat(*charts)


alt.VConcatChart(...)

In [85]:
ch = alt.Chart(df).mark_line(point=True).encode(
    alt.Y('sum(Profit)'),
    alt.X('month(Order Date)'),
    column = 'year(Ship Date)'
).resolve_scale(y = 'independent',x = 'independent')
ch

alt.Chart(...)